## Step 1: Clone the Repository

In [ ]:
!git clone https://github.com/ab-2109/HMRAG.git
%cd HMRAG

## Step 2: Install Dependencies

In [ ]:
# Install required packages
# Handle dependency conflicts by installing core packages first
import os

print("Installing dependencies (this may take a few minutes)...")

# Install numpy first to avoid conflicts
!pip install -q numpy==1.26.4

# Install core dependencies with compatible versions
!pip install -q --no-deps lightrag-hku
!pip install -q langchain langchain-community langchain-core
!pip install -q transformers torch tqdm ollama
!pip install -q google-search-results
!pip install -q networkx aiohttp tenacity tiktoken

# Install optional vision dependencies if needed
# Uncomment if you need vision model support (Qwen2.5-VL)
# !pip install -q qwen_vl_utils opencv-python

print("✓ Dependencies installed successfully!")
print("Note: Some dependency warnings are normal and won't affect functionality.")

## Step 3: Install and Setup Ollama (Required for LLM)
Note: Ollama is required for this system. On Colab, follow these steps to install it.

In [ ]:
# Install Ollama on Colab
print("Installing zstd (required for Ollama)...")
!sudo apt-get update -qq
!sudo apt-get install -y -qq zstd

print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# Verify installation
import subprocess
import time
import os

# Check if ollama is installed
try:
    result = subprocess.run(['which', 'ollama'], capture_output=True, text=True)
    ollama_path = result.stdout.strip()
    
    if not ollama_path:
        print("⚠️ Ollama not found in PATH. Trying common locations...")
        if os.path.exists('/usr/local/bin/ollama'):
            ollama_path = '/usr/local/bin/ollama'
        elif os.path.exists('/usr/bin/ollama'):
            ollama_path = '/usr/bin/ollama'
        else:
            raise FileNotFoundError("Ollama binary not found")
    
    print(f"✓ Ollama found at: {ollama_path}")
    
    # Start Ollama server in background
    print("Starting Ollama server...")
    ollama_process = subprocess.Popen(
        [ollama_path, 'serve'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    time.sleep(10)  # Wait for server to start
    
    # Pull required models
    print("Pulling qwen2.5:7b model (this may take 5-10 minutes)...")
    !{ollama_path} pull qwen2.5:7b
    
    print("Pulling nomic-embed-text model...")
    !{ollama_path} pull nomic-embed-text
    
    print("✓ Ollama setup complete!")
    
except Exception as e:
    print(f"❌ Error setting up Ollama: {e}")
    print("You may need to restart the runtime and try again.")

## Step 4: Setup API Keys

In [ ]:
# Set your API keys here
import os
from google.colab import userdata

# Store secrets in Colab's secret manager (left sidebar -> Key icon)
# Then access them like this:
try:
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
except:
    # Or set directly (not recommended for production)
    SERPAPI_API_KEY = "your-serpapi-key-here"

print("API keys configured!")
print("Note: OpenAI API key is not required. System uses Ollama for LLM inference.")

## Step 5: Download ScienceQA Dataset

In [ ]:
# Download the ScienceQA dataset
!bash dataset/download_ScienceQA.sh

## Step 6: Create Required Directories

In [ ]:
# Create output and working directories
!mkdir -p outputs
!mkdir -p lightrag_workdir

# Check dataset structure
!ls -la dataset/

## Step 7: Run Inference (Small Test)
Start with a small test run (5 examples)

In [ ]:
# Run on a small subset first to test
!python3 main.py \
    --data_root ./dataset/ScienceQA/data \
    --image_root ./dataset/ScienceQA/images \
    --output_root ./outputs \
    --caption_file ./dataset/ScienceQA/captions.json \
    --working_dir ./lightrag_workdir \
    --serpapi_api_key "$SERPAPI_API_KEY" \
    --test_split test \
    --test_number 5 \
    --shot_number 0 \
    --label test_run \
    --save_every 5

## Step 8: Run Full Inference
After testing, run on the full dataset

In [ ]:
# Full inference run
!python3 main.py \
    --data_root ./dataset/ScienceQA/data \
    --image_root ./dataset/ScienceQA/images \
    --output_root ./outputs \
    --caption_file ./dataset/ScienceQA/captions.json \
    --working_dir ./lightrag_workdir \
    --serpapi_api_key "$SERPAPI_API_KEY" \
    --test_split test \
    --shot_number 2 \
    --label full_run \
    --save_every 50 \
    --use_caption

## Step 9: View Results

In [ ]:
# View output files
!ls -lh outputs/

# Load and display results
import json

with open('outputs/test_run_test.json', 'r') as f:
    results = json.load(f)

print(f"Total results: {len(results)}")
print("\nSample results:")
for qid, answer in list(results.items())[:5]:
    print(f"Question ID: {qid}, Answer: {answer}")

## Alternative: Use OpenAI Models Directly
If Ollama setup is difficult, modify the agents to use OpenAI API directly

## Download Results to Local Machine

In [ ]:
# Download results
from google.colab import files

# Download the results file
files.download('outputs/test_run_test.json')

# Or zip and download all outputs
!zip -r outputs.zip outputs/
files.download('outputs.zip')